In [ ]:

import os
import numpy as np
import glob
import cv2
import imageio


import tensorflow
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Dropout, MaxPooling2D, Activation

import random
random.seed(42)

import wandb

from wandb.keras import WandbCallback

# !wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip
# !unzip -q '/content/nature_12K.zip'

from tqdm import tqdm




In [ ]:
sweep_config = {
    'method': 'random', 
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'kernel_size':{
            'values': [[(3,3),(3,3),(3,3),(3,3),(3,3)], 
            [(3,3),(5,5),(5,5),(7,7),(7,7)], 
            [(7,7),(7,7),(5,5),(5,5),(3,3)], 
            [(3,3),(5,5),(7,7),(9,9),(11,11)] ]
        },
        'weight_decay': {
            'values': [0, 0.0005, 0.005]
        },
        'dropout': {
            'values': [0, 0.2, 0.4]
        },
        'learning_rate': {
            'values': [1e-3, 1e-4]
        },
        'activation': {
            'values': ['relu', 'elu', 'selu']
        },
        'batch_norm':{
            'values': ['true','false']
        },
        'filt_org':{
            'values': [[32,32,32,32,32],[32,64,64,128,128],[128,128,64,64,32],[32,64,128,256,512]]
        },
        'data_augment': {
            'values': ['true','false']
        },
        'bs': {
            'values': [32, 16]
        },
        'num_dense':{
            'values': [64, 128, 256, 512]
        },
        'epochs':{
            'values': [5,10,15]
        },
    }
}

img_size_input = 128
ClassesId = 10

In [ ]:
def train(config = sweep_config):

    with wandb.init(project="Assignment2", entity="cs6910jananiteam", config=config):
      config = wandb.init().config
      wandb.run.name = 'num_dense_'+ str(config.num_dense)+ \
                          '_bs_'+str(config.bs)+ \
                          '_ac_'+ str(config.activation )+ \
                          '_epochs_' + str(config.epochs)+ \
                          '_filter_' + str(config.filt_org )+ \
                          '_lr_'+ str(config.learning_rate )+ \
                          '_aug_' + str(config.data_augment)+ \
                          '_drop_' + str(config.dropout)+ \
                          '_kernal_' + str(config.kernel_size)
      
      input_shape = (img_size_input, img_size_input , 3)
      model = Sequential()

      filter = config.filt_org
      model.add(Conv2D(filters = filter[0], kernel_size = config.kernel_size[0],padding = 'same', 
                      input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))

      if config.activation == "relu":
          model.add(Activation('relu'))
      elif config.activation == "elu":
          model.add(Activation('elu'))
      elif config.activation == "selu":
          model.add(Activation('selu'))

      if config.batch_norm == 'True':
          model.add(BatchNormalization())

      model.add(MaxPooling2D(pool_size=(2, 2)))

      model.add(Conv2D(filters = filter[1], kernel_size = config.kernel_size[1], padding = 'same', 
                      input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))

      if config.activation == "relu":
          model.add(Activation('relu'))
      elif config.activation == "elu":
          model.add(Activation('elu'))
      elif config.activation == "selu":
          model.add(Activation('selu'))

      if config.batch_norm == 'True':
          model.add(BatchNormalization())

      model.add(MaxPooling2D(pool_size=(2, 2)))

      model.add(Conv2D(filters = filter[2], kernel_size = config.kernel_size[2], padding = 'same', 
                      input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))

      if config.activation == "relu":
          model.add(Activation('relu'))
      elif config.activation == "elu":
          model.add(Activation('elu'))
      elif config.activation == "selu":
          model.add(Activation('selu'))

      if config.batch_norm == 'True':
          model.add(BatchNormalization())

      model.add(MaxPooling2D(pool_size=(2, 2)))

      model.add(Conv2D(filters = filter[3], kernel_size = config.kernel_size[3], padding = 'same', 
                      input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))

      if config.activation == "relu":
          model.add(Activation('relu'))
      elif config.activation == "elu":
          model.add(Activation('elu'))
      elif config.activation == "selu":
          model.add(Activation('selu'))

      if config.batch_norm == 'True':
          model.add(BatchNormalization())

      model.add(MaxPooling2D(pool_size=(2, 2)))
      
      model.add(Conv2D(filters = filter[4], kernel_size = config.kernel_size[4], padding = 'same', 
                      input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))

      if config.activation == "relu":
          model.add(Activation('relu'))
      elif config.activation == "elu":
          model.add(Activation('elu'))
      elif config.activation == "selu":
          model.add(Activation('selu'))

      if config.batch_norm == 'True':
          model.add(BatchNormalization())

      model.add(MaxPooling2D(pool_size=(2, 2)))

      model.add(Flatten())
      model.add(Dense(config.num_dense, activation = config.activation, kernel_regularizer = regularizers.l2(config.weight_decay)))
      model.add(Dropout(config.dropout))
      model.add(BatchNormalization())

      model.add(Dense(ClassesId, activation = "softmax"))

      optimizer = Adam(lr=config.learning_rate, beta_1=0.9, beta_2=0.999)
      
      model.compile(loss = "categorical_crossentropy", optimizer = optimizer, metrics=['accuracy'])

      #data augmentation
      if config.data_augment == 'true':
          datagen = ImageDataGenerator(
              rotation_range=45,  
              height_shift_range=0.1,  
              horizontal_flip=True, 
              vertical_flip=True, 
              validation_split=0.2
          )
      else:
          datagen = ImageDataGenerator(rescale = 1.0, validation_split=0.2)

      # datagen.fit(X_train)
      TrainData = datagen.flow_from_directory(
              directory='/content/inaturalist_12K/train',
              target_size = (img_size_input, img_size_input),
              bs=config.bs,
              class_mode="categorical",
              shuffle=True,
              subset='training',
              seed=22)
      
      ValidationData = datagen.flow_from_directory(
              directory='/content/inaturalist_12K/train',
              target_size = (img_size_input, img_size_input),
              bs=config.bs,
              class_mode="categorical",
              shuffle=True,
              subset='validation',
              seed=22)
      
      model.fit_generator(
          TrainData,
          steps_per_epoch = TrainData.samples // config.bs,
          epochs = config.epochs,
          verbose = 1,
          validation_data = ValidationData,
          validation_steps = ValidationData.samples // config.bs,
          callbacks = [WandbCallback()]
      )


In [ ]:

sweep_id = wandb.sweep(sweep_config,project="Assignment2", entity="cs6910jananiteam")
wandb.agent(sweep_id, function = train, count = 30)